# Rotationally equivariant graph neural networks for integrated circuits

This file contains some rough notes on how you might apply rotational equivariance to
a graph neural network (GNN) model describing an integrated circuit (IC).
Any (scalar) predictions you make using a GNN should not change if you rotate the IC by 90 degrees in the XY plane.  But if you use an ordinary GNN model, this is not gauranteed.

This document explains how to exploit this 90-degree rotational symmetry to improve the accuracy of a GNN model.

## Why bother?

Many models are rotationally inviariant.  In general, exploiting symmetry can improve model performace and robustness.  Equivariance is commonly used to improve the accuracy of molecular models *(eg. AlphaFold2)* and CNNs.  Here is a visually striking example of the benefits of applying equivariance to CNNs:

### Example 1
The output of a conventional CNN is not rotationally equivariant.

https://github.com/QUVA-Lab/escnn/raw/master/visualizations/conventional_cnn.gif

### Example 2
Output of an equivariant CNN

https://github.com/QUVA-Lab/escnn/raw/master/visualizations/vectorfield.gif

*(Credit: Maurice Weiler, Gabriele Cesa)*

## GNN notation

![simple_graph_with_edge_attributes](./images/simple_graph_with_edge_attributes.svg)

General form of the update rule that most GNNs use to update their node attributes:

$h^\prime_i\ =\ \mathcal{U}\left(h_i \ , \ \bigoplus\limits_{j\in \mathcal{N}(i)}\ \mathcal{m}(h_i,\ h_j,\ e_{ij}) \right)$

Where:
 - $h_i$  are node attributes (for circuit components, their terminals, and points of interest along the wires in the circuit).

 - $e_{ij}$  are (optional) edge attributes between nodes $i$ and $j$.  ($e_{ij}$ might encode whether nodes $i$ and $j$ are nearby and/or directly connected by a wire.)

 - $\mathcal{N}(i)$ are the neighbors of node $i$.

 - $\bigoplus$  is a message aggregator *(typically a sum, $\sum_{j\in \mathcal{N}(i)}$)*

 - $\mathcal{m}()$  calculates the message from node $j$ to node $i$.  *(Typically an MLP.)*

 - $\mathcal{U}()$  is the node update function, combining the aggregated messages with the attributes for node $i$.  *(Typically an MLP.)*

## Introducing geometry
In general, the 3D coordinates of each node ($\vec{x}_i, \vec{x}_j$) could affect each node and it's communication with it's neighbors.  Let's make this position dependence explicit:

$h^\prime_i\ =\ \mathcal{U}\left(h_i \ , \ \bigoplus\limits_{j\in \mathcal{N}(i)}\ \mathcal{m}(h_i,\ h_j,\ e_{ij},\ \vec{x}_i,\ \vec{x}_j) \right)$

Where
- $\vec{x}_i$ is the 3D position of node $i$.
- It's convenient to assume that $h_i$ and $e_{ij}$ do not *initially* contain position information *(but may acquire it later through interaction with neighbors)*. 


## Translational symmetry

Suppose only the relative position of nodes matters.  This reduces the complexity of the model because it reduces the number of arguments we need to pass to the message function:

$h^\prime_i\ =\ \mathcal{U}\left(h_i \ , \ \bigoplus\limits_{j\in \mathcal{N}(i)} \mathcal{m}(h_i,\ h_j,\ e_{ij},\  \vec{x}_i-\vec{x}_j) \right)$


To simplify the notation, let's borrow the notation used in Convolutional Neural Networks:

$h^\prime_i\ =\ \mathcal{U}\left(\sum\limits_{j\in \mathcal{N}(i)} k(\vec{x}_j-\vec{x}_i)\ h_j \right)$

Where
- $k(\vec{x}_j-\vec{x}_i)$ is a message function which only depends on the relative physical location of nodes $i$ and $j$.  *(Also called a "convolution kernel".  This terminology is borrowed from CNNs, which are GNNs for graphs of regularly spaced nodes on a lattice.)*  It could return a scalar or a matrix.  *(We will consider more general message functions later.)*
- $\mathcal{N}(i)$ denotes the neighbors of node $i$.  *(For CNNs, $\mathcal{N}(i)$ would represent all the nodes located a square window around node $i$, as shown below.)*

![CNN_neighbors_of_i](./images/CNN_neighbors_of_i.svg)

NOTE: The equation above no longer describes a general graph neural network.
But using simple notation now will help us later.
*(More general graph neural networks are discussed at the end of this document.)*

## Rotational Symmetry

We want to gaurantee that the predictions of the GNN are consistent, regardless of the orientation of the input graph.
ICs are typically manufactured placing wires aligned with the X or Y directions of the wafer.  So our GNN model should be behave consistently when applied to the same graphs rotated by 90 degrees..


## Data augmentation and pooling

*One way* to ensure this is using *data augmentation* and *pooling*.

- *During training*, we can augment our training data set.  We can make 4 copies each graph in the training set by rotating the coordinates by 0, 90, 180, and 270 degrees, and use these rotated graphs for training.
- At *inference time*, we *could* do the same thing: We could rotate the input graph 4 times and make predictions from each rotated version of the graph.  At the end of the calculation, the conclusions from all 4 versions of the graph would be *pooled* into a single answer *(see below)*.  This causes a 4x increase in computational burden *during inference*, but the resulting predictions are gauranteed to be consistent ("rotationally equivariant") regardless of the orientation of the input graph.

This is the simplest way to implement equivariance.  But it is not the most powerful.

## Lifting

So far, we have treated the rotated graphs that we get from data augmentation as independent graphs.  To improve the expressive power of the GNN model, perhaps these 4 rotated graphs could interact with each other in some way?

At this point it is useful to introduce the concept of a "lifted" graph.

![simple_graph](./images/simple_graph.svg)  *...becomes a "lifted graph" $\ \longrightarrow \ $*  ![lifted_graph_mu_nu](./images/lifted_graph_mu_nu.svg)
*(I will add the edges later...)*

### Definition: Lifted Graphs
A "lifted" graph (shown on the right) contains 4 indentical copies of the original graph.  Each node in the *lifted* graph, $\nu$, *(denoted by dark dots in the figure on the right),* corresponds to a rotated version of the original graph.  To compute the attributes/embeddings of the nodes in the lifted graph (denoted $\nu$), the coordinates ($\vec{x}_i$) of all the nodes in the input graph, $i$, are rotated beforehand by a corresponding angle, $\theta_\nu$, which is one of 4 possible angles ($0, \frac{\pi}{2}, \pi, \frac{3}{2}\pi$).

Notation:
- $\theta_\nu$  is the orientation corresponding to node $\nu$ *(where $\theta_\nu \in \{0, \frac{pi}{2}, \pi, \frac{3}{2}\pi\}$)*.
- $\mathsf{h}_{\nu}$ = The attributes of node $\nu$.  (If position dependent, then we assume these attributes are calculated assuming that the physical location of node $\nu$ was initially rotated by $\theta_\nu$.)


### Communication between nodes in lifted graphs

### Simple equivariance: Independent lifted graphs

During training, we *could* let data propogate through each rotated version of the original graph *independently*.  In other words, the coordinates of each input graph would initially be rotated and loaded into the corresponding nodes ($\nu$) of the *lifted* graph.  But as data flows through the network, nodes ($\nu, \mu$) corresponding to different orientations of the graph ($\theta_\nu \neq \theta_\mu$) *would not* talk to each other.  But they would all share the same message ($k()$) and update ($\mathcal{U}()$) functions *(which are trained using all 4 graphs)*.  At inference, the predictions from each of the 4 graphs would be calculated and pooled *(see below)*.  *(This is equivalent to the "data augmentation + pooling" method discussed above.)*

![lifted_graph_independent](./images/lifted_graph_independent.svg)

### Full equivariance: Interdependent lifted graphs

*More generally*, we can improve the performace of the model by allowing the nodes from *different orientations* to talk to each other during the node update process.

![lifted_graph_v1](./images/lifted_graph_v1.svg)

To do that, we must define a more general message passing function (convolution kernel).

### Definition $\mathsf{k}(\vec{x},\theta)$ 

$\mathsf{k}(\vec{x},\theta)$ is a convolution kernel which is defined over the space of translations ($\vec{x}$) *and* rotations ($\theta$).

However to satisfy *rotational equivariance*, this new kernel must obey *rotational* and *translational* symmetry. 
That means it must only depend on
the difference between the node positions
and the difference between the two angles
$(\theta_\nu-\theta_\mu)$.  *(See below.)*


### Update rule for node $\nu$

![lifted_graph_v2.svg](./images/lifted_graph_v2.svg)

$\mathsf{h}^\prime_\nu\ =\ \mathcal{U}\left(\sum\limits_{\mu\in\mathcal{N}(\nu)} \mathsf{k}(R_{\theta_\nu}(\vec{x}_{j(\mu)}-\vec{x}_{i(\nu)}),\ \theta_\mu-\theta_\nu) \ \mathsf{h}_\mu \right)$

Where
- $R_{\theta_\nu}$ is the rotation matrix corresponding to angle $\theta_\nu$
- $i(\nu)$ is the index of node $i$ from the original graph corresponding to node $\nu$ in the "lifted" graph.

***Links to videos that explain the justification for this formula in more detail are provided below.***

**Clarification:** The coordinates that appear in the equation above ($\vec{x}_{i(\nu)}$) represent the *unrotated* coordinates of node $i(\nu)$ from the original graph.  In this equation, we rotated the coordinates $(\vec{x}_{j(\mu)}-\vec{x}_{i(\nu)})$ by the angle $\theta_\nu$ because this is the update rule for node $\nu$.  


### Pooling over orientations
*If the features we want to learn from the graph are scalars (eg. power, delay, impedance),* then at the end of the computation, we can combine the embeddings at all orientations.  At the node level, it might look like this:

$h_i^{(final)}\ =\sum\limits_{\theta\in\{0, \frac{pi}{2}, \pi, \frac{3}{2}\pi\}} \mathsf{h}_{\nu(i,\theta)}$

Where:
- $\nu(i,\theta)$ is the index of node $\nu$ (from the "lifted" graph) corresponding to node $i$ (from the original graph) initially rotated by $\theta$.

Vector quantities (eg. fields) can be pooled also, but the contribution from each rotated graph must be rotated back before being added together.  (Otherwise they will cancel each other out.)


### Model complexity
Model complexity is 4x as large the same as the original, non-equivariant GNN because the new message function $\mathsf{k}(\vec{x},\theta)$ depends explicitly on $\theta$, which can have 4 values.

In spite of this, the method has been reported to perform better than a traditional GNN model with the same computational complexity *(ie. with the same number of parameters)*, trained using data-augmentation.

## General equivariant GNNs with $C_4$ symmetry

The update function above ignores edge attributes and assumes that nodes are updated using a simple linear convolution kernel, $k()$.

As promised, here's a more general version of the equivariant GNN node update function:

$\mathsf{h}^\prime_\nu\ =\ \mathcal{U}\left(\mathsf{h}_\nu\ ,\ \bigoplus\limits_{\mu\in\mathcal{N}(\nu)}  \ \mathcal{m}\left(h_{\nu(i)},\ h_{\mu(j)},\ e_{i(\nu),j(\mu)},\ R_{\theta_\nu}(\vec{x}_{j(\mu)}-\vec{x}_{i(\nu)}),\ \theta_\mu-\theta_\nu\right) \right)$


*(I have assumed that the edge attributes $e_{i,j}$ are independent of position of the nodes and are rotationally invariant.)*

## *Further reading*

These notes were inspired by Erik Bekker's videos on Equivariant CNNs:

- lecture 1.1  https://www.youtube.com/watch?v=z2OEyUgSH2c&list=PL8FnQMH2k7jzPrxqdYufoiYVHim8PyZWd&index=1
- lecture 1.2  https://www.youtube.com/watch?v=F0OxOCZwm1Q&list=PL8FnQMH2k7jzPrxqdYufoiYVHim8PyZWd&index=2
- lecture 1.3  https://www.youtube.com/watch?v=cWG_1IzI0uI&list=PL8FnQMH2k7jzPrxqdYufoiYVHim8PyZWd&index=3
- lecture 1.4  https://www.youtube.com/watch?v=X3gP1voalDE&list=PL8FnQMH2k7jzPrxqdYufoiYVHim8PyZWd&index=4
- lecture 1.5  https://www.youtube.com/watch?v=kTvow5-eCCQ&list=PL8FnQMH2k7jzPrxqdYufoiYVHim8PyZWd&index=5
- lecture 1.6  https://www.youtube.com/watch?v=mntjPJYxwTI&list=PL8FnQMH2k7jzPrxqdYufoiYVHim8PyZWd&index=6
- lecture 1.7  https://www.youtube.com/watch?v=erlCaoj6sTg&list=PL8FnQMH2k7jzPrxqdYufoiYVHim8PyZWd&index=7

After watching those you can skip to the video on equivariant graph neural networks:
- lecture 3.2  https://www.youtube.com/watch?v=o-KcYASwUco&list=PL8FnQMH2k7jzPrxqdYufoiYVHim8PyZWd&index=17

*(You can skip the videos on steerable neural networks (2.1-2.7) because they are only relevant for systems
with continuous rotational symmetry.)*